In [582]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from crowdnalysis.cmdstan import StanMultinomialOptimizeConsensus, \
        StanMultinomialEtaOptimizeConsensus, \
        StanDSOptimizeConsensus, \
        StanDSEtaOptimizeConsensus, \
        StanDSEtaHOptimizeConsensus
from crowdnalysis.dawid_skene import DawidSkene
from scipy.stats import entropy
import numpy as np
import json


ImportError: cannot import name 'StanMultinomialOptimizeConsensus' from 'crowdnalysis.cmdstan' (/home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/__init__.py)

In [598]:
ds = DawidSkene()
stan_ds = StanDSOptimizeConsensus()
stan_dseta = StanDSEtaOptimizeConsensus()
stan_dsetah = StanDSEtaHOptimizeConsensus()

In [599]:
def generate_pi(k, noise):
    _pi = np.zeros((k,k))
    for _k in range(k):
        _pi[_k] = np.random.dirichlet(noise*np.ones(k), size=1)[0]
        while np.max(np.delete(_pi[_k],_k)) >= _pi[_k,_k]:
            _pi[_k] = np.random.dirichlet(noise*np.ones(k), size=1)[0]
    return _pi    

In [600]:
generate_pi(3,10)

array([[0.43901526, 0.24473236, 0.31625238],
       [0.20063228, 0.41688836, 0.38247936],
       [0.31532128, 0.33947469, 0.34520403]])

In [601]:
def generate_pis(w, k, noise):
    _pi = np.zeros((w,k,k))
    for _w in range(w):
        _pi[_w] = generate_pi(k, noise)
    return _pi


In [699]:
t = 100
num_annotations_per_task = 3
w = 20
real_tau = np.array([0.1, 0.3, 0.6])
k = len(real_tau)
noise = 0.5
real_pi = generate_pis(w,k,noise)
real_parameters = {'tau':real_tau, 'pi':real_pi}
print(real_parameters)
t_C, m = ds.sample(t, num_annotations_per_task, parameters = real_parameters)

{'tau': array([0.1, 0.3, 0.6]), 'pi': array([[[7.42563510e-01, 1.90345381e-01, 6.70911085e-02],
        [2.30592312e-01, 7.19389407e-01, 5.00182813e-02],
        [1.93550910e-01, 1.55352956e-02, 7.90913794e-01]],

       [[7.13459256e-01, 5.52989310e-03, 2.81010851e-01],
        [7.97592655e-02, 9.09328709e-01, 1.09120252e-02],
        [2.25728639e-02, 7.29374761e-02, 9.04489660e-01]],

       [[7.37057595e-01, 2.58627244e-01, 4.31516040e-03],
        [3.43114660e-01, 6.30222109e-01, 2.66632310e-02],
        [2.41874302e-01, 2.98602537e-01, 4.59523161e-01]],

       [[5.78193428e-01, 1.99180735e-02, 4.01888499e-01],
        [6.04347364e-03, 9.82195442e-01, 1.17610841e-02],
        [3.91897889e-03, 4.40892210e-01, 5.55188811e-01]],

       [[5.27521533e-01, 2.94571950e-01, 1.77906518e-01],
        [3.17204530e-01, 4.45231616e-01, 2.37563853e-01],
        [3.00411977e-01, 2.85525396e-01, 4.14062627e-01]],

       [[6.01140886e-01, 1.93939454e-01, 2.04919660e-01],
        [1.61772611e-04,

In [700]:
m

array([[ 0, 12,  2],
       [ 0,  7,  1],
       [ 0,  3,  1],
       [ 1, 16,  1],
       [ 1, 15,  1],
       [ 1, 17,  2],
       [ 2,  4,  2],
       [ 2,  8,  1],
       [ 2,  0,  2],
       [ 3,  2,  0],
       [ 3, 14,  1],
       [ 3,  5,  1],
       [ 4,  4,  0],
       [ 4, 19,  0],
       [ 4, 19,  1],
       [ 5,  3,  2],
       [ 5, 19,  1],
       [ 5, 14,  2],
       [ 6, 17,  0],
       [ 6,  5,  2],
       [ 6, 16,  1],
       [ 7,  8,  1],
       [ 7, 15,  1],
       [ 7,  3,  1],
       [ 8, 18,  1],
       [ 8,  1,  2],
       [ 8,  9,  2],
       [ 9,  1,  1],
       [ 9,  8,  0],
       [ 9, 10,  1],
       [10, 10,  2],
       [10, 13,  1],
       [10, 17,  2],
       [11, 10,  0],
       [11,  0,  0],
       [11, 17,  0],
       [12, 17,  0],
       [12, 17,  2],
       [12, 10,  1],
       [13,  0,  2],
       [13,  6,  2],
       [13, 16,  0],
       [14, 17,  0],
       [14,  7,  2],
       [14, 12,  2],
       [15, 11,  2],
       [15,  8,  1],
       [15,  

In [701]:
learned_t_C, learned_parameters = ds.m_fit_and_compute_consensus(m, t, k, w)

In [702]:

learned_t_C_stan, learned_parameters_stan = stan_ds.m_fit_and_compute_consensus(m, t, k, w)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/DS.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [703]:

learned_t_C_eta_stan, learned_parameters_eta_stan = stan_dsetah.m_fit_and_compute_consensus(m, t, k, w)
print(learned_parameters_eta_stan)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/DSEtaH.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.0793383, 0.247531 , 0.673131 ]), 'eta': array([[10.7435  ,  3.30404 ],
       [ 1.41556 ,  3.35297 ],
       [ 1.99463 ,  0.690085]]), 'pi': array([[[9.75303e-01, 1.70983e-05, 2.46798e-02],
        [2.93792e-01, 6.86581e-01, 1.96270e-02],
        [1.21661e-01, 7.90361e-02, 7.99303e-01]],

       [[9.70451e-01, 1.93867e-05, 2.95292e-02],
        [2.68049e-01, 7.07026e-01, 2.49256e-02],
        [3.09120e-02, 2.68307e-01, 7.00781e-01]],

       [[9.64248e-01, 2.28736e-05, 3.57296e-02],
        [1.89775e-01, 8.01985e-01, 8.24024e-03],
        [1.44295e-01, 5.26617e-01, 3.29088e-01]],

       [[9.70622e-01, 2.74825e-05, 2.93510e-02],
        [9.06291e-02, 8.78152e-01, 3.12189e-02],
        [9.95022e-02, 4.17271e-01, 4.83227e-01]],

       [[9.64301e-01, 2.26700e-05, 3.56766e-02],
        [2.87545e-01, 7.00669e-01, 1.17855e-02],
        [2.49619e-01, 2.52362e-01, 4.98019e-01]],

       [[9.64403e-01, 2.19227e-05, 3.55754e-02],
        [1.02802e-01, 8.78210e-01, 1.89886e-02],

In [704]:
def err_rate(t_C, t_C_m):
    max_t_C_m = np.argmax(t_C_m,axis=1)
    #print(max_t_C_m)
    return 1-(np.sum(t_C==max_t_C_m)/len(t_C))
def log_score(t_C, t_C_m):
    f = t_C_m.flatten()
    start = k*np.arange(t)
    return np.sum(np.log(f[start+t_C]))

def evaluate_ds_model(real_parameters, learned_parameters, w, k):
    d = {}
    d["tau_kl"] = entropy(learned_parameters["tau"],qk=real_parameters["tau"], base=2)
    d["pi_kl"] = np.zeros((w, k))
    for _w in range(w):
        for _k in range(k):
             d["pi_kl"][_w,_k] = entropy(learned_parameters["pi"][_w,_k],qk=real_parameters["pi"][_w,_k], base=2)
    return d
            

In [705]:
print("err rate:", err_rate(t_C,learned_t_C))
print("log score:", log_score(t_C, learned_t_C))
r = evaluate_ds_model(real_parameters, learned_parameters, w, k)
print (r)
print(np.sum(r["pi_kl"]))

err rate: 0.31999999999999995
log score: -94.74458194001244
{'tau_kl': 0.04517487516520644, 'pi_kl': array([[6.57887471e-01, 9.92544496e-01, 2.11559433e-01],
       [1.67416040e+00, 1.38416715e+00, 3.62586271e-01],
       [1.66749183e+00, 1.82532777e-01, 2.93264815e-01],
       [1.05098128e+00, 2.60605452e+00, 7.59656505e-01],
       [1.10382834e-01, 6.27111287e-02, 3.85896176e-03],
       [2.52305752e-01, 1.43901195e+00, 9.61419428e-01],
       [5.77071955e-02, 4.45286326e+00, 7.03169913e-02],
       [4.08453923e-02, 8.23927626e-02, 1.59714275e-01],
       [1.81202081e+00, 1.90910142e+00, 8.14421871e-01],
       [3.30486446e-01, 3.42281526e-01, 6.81547754e-01],
       [2.08766787e+00, 1.03388480e-01, 1.18750888e-01],
       [1.85813876e+00, 4.39695769e-02, 5.04175019e-01],
       [3.60338279e+00, 4.53616607e-01, 9.05046194e-02],
       [1.62740245e-01, 1.84472909e+00, 1.00629600e+00],
       [2.30519680e-01, 2.93957369e-01, 7.35529399e-02],
       [3.26948548e+00, 2.84765631e+00, 8.54

In [706]:
print("err rate:", err_rate(t_C,learned_t_C_stan))
print("log score:", log_score(t_C, learned_t_C_stan))
r = evaluate_ds_model(real_parameters, learned_parameters_stan, w, k)
print (r)
print(np.sum(r["pi_kl"]))

err rate: 0.16000000000000003
log score: -382.2538512808943
{'tau_kl': 0.049813883025452095, 'pi_kl': array([[0.42941364, 0.09479133, 0.3384076 ],
       [0.48709698, 0.09833842, 0.14482404],
       [0.44015065, 0.30986821, 0.30832551],
       [0.78946517, 0.20762203, 0.57223468],
       [0.92269802, 0.5118952 , 0.41963405],
       [0.73422486, 0.21066968, 0.02503594],
       [1.07558355, 0.50636588, 0.18157866],
       [0.7717165 , 0.46493582, 0.73085267],
       [0.07748044, 0.19409061, 0.12717876],
       [0.61208776, 0.45564078, 0.14981682],
       [0.06612615, 0.22424888, 0.28582044],
       [0.22249173, 0.73484418, 0.06021906],
       [0.15253375, 0.33049285, 0.03585656],
       [0.82582618, 0.12224383, 0.16152596],
       [1.06403521, 0.55812014, 0.64330512],
       [0.01787712, 0.05291439, 0.54261403],
       [0.25568282, 0.18307971, 0.04268343],
       [0.54990622, 0.28530939, 0.21406338],
       [0.03158938, 0.12923944, 0.19244882],
       [0.34379779, 0.36829848, 0.41580069]

In [707]:
print("err rate:", err_rate(t_C,learned_t_C_eta_stan))
print("log score:", log_score(t_C, learned_t_C_eta_stan))
r = evaluate_ds_model(real_parameters, learned_parameters_eta_stan, w, k)
print (r)
print(np.sum(r["pi_kl"]))

err rate: 0.19999999999999996
log score: -79.48790408366253
{'tau_kl': 0.016543251503508858, 'pi_kl': array([[0.3477844 , 0.0299405 , 0.11616743],
       [0.33457032, 0.24178209, 0.260225  ],
       [0.48242174, 0.1027581 , 0.16500684],
       [0.61433014, 0.25615726, 0.33436667],
       [0.75617516, 0.36656944, 0.02099459],
       [0.5675029 , 0.9238522 , 0.36412181],
       [0.9310727 , 0.86066066, 0.00519241],
       [0.63998663, 0.66257056, 0.28516614],
       [0.04222196, 0.13372601, 0.29248999],
       [0.48596709, 0.45159239, 0.14771802],
       [0.03055394, 0.52380679, 0.0277592 ],
       [0.23287959, 0.45785861, 0.28860552],
       [0.23019667, 0.58244736, 0.02633253],
       [0.650026  , 0.11426103, 0.42768368],
       [0.87839244, 0.40625062, 0.04202895],
       [0.03985366, 0.49370854, 0.01221436],
       [0.14898531, 0.47362666, 0.16955326],
       [0.49724278, 0.17605295, 0.1519061 ],
       [0.00781576, 0.55356864, 0.01017327],
       [0.20349368, 0.28584917, 0.02419634]

In [708]:

def json_dump(fp, m, w, t, k):
    t_A = m[:,0]+1
    w_A = m[:,1]
    ann = m[:,2]+1
    a = m.shape[0]
    
    json_data = {'w': w,
             't': t,
             'a': a,
             'k': k,
             't_A': t_A.tolist(),
             'w_A': w_A.tolist(),
             'ann': ann.tolist()}
    json.dump(json_data, fp, indent=2)


In [496]:
with open('DS.json', 'w') as outfile:
    json_dump(outfile, m, w, t, k)

In [26]:
print(a)
print(m[9980:10000], t_A[9980:])

5000
[] []


In [27]:

sm2oc = StanMultinomial2OptimizeConsensus()
t_C, d = sm2oc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial2.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.232108, 0.307849, 0.460043]),
 'eta': array([[0.0235701, 0.574529 ],
        [1.36155  , 0.995963 ],
        [0.717564 , 0.681768 ]]),
 'pi': array([[0.393751, 0.384579, 0.22167 ],
        [0.157639, 0.615146, 0.227215],
        [0.244745, 0.253665, 0.50159 ]]),
 'log_p_t_C': array([[-7.89146, -6.59551, -5.58984],
        [-6.19144, -6.33072, -7.70673],
        [-6.71883, -8.68824, -7.06076],
        ...,
        [-6.19144, -6.33072, -7.70673],
        [-7.86789, -7.95706, -5.62563],
        [-7.31693, -6.9611 , -6.3074 ]]),
 't_C': array([[0.0682834, 0.249539 , 0.682178 ],
        [0.478531 , 0.416314 , 0.105155 ],
        [0.540559 , 0.0754293, 0.384012 ],
        ...,
        [0.478531 , 0.416314 , 0.105155 ],
        [0.0882671, 0.0807367, 0.830996 ],
        [0.193361 , 0.275997 , 0.530643 ]])}

In [28]:
t_C, d = smoc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.0298387, 0.528094 , 0.442067 ]),
 'pi': array([[0.679218, 0.214762, 0.10602 ],
        [0.263376, 0.264884, 0.47174 ],
        [0.210426, 0.562761, 0.226813]]),
 'log_p_t_C': array([[-13.3208 ,  -5.54939,  -6.41698],
        [ -8.90025,  -7.29222,  -5.65824],
        [ -8.45474,  -6.72079,  -7.55069],
        ...,
        [ -8.90025,  -7.29222,  -5.65824],
        [-12.1694 ,  -5.5551 ,  -7.4007 ],
        [-11.4635 ,  -6.13223,  -6.49197]]),
 't_C': array([[2.96840e-04, 7.04035e-01, 2.95668e-01],
        [3.16675e-02, 1.58115e-01, 8.10218e-01],
        [1.09498e-01, 6.20086e-01, 2.70415e-01],
        ...,
        [3.16675e-02, 1.58115e-01, 8.10218e-01],
        [1.15684e-03, 8.62611e-01, 1.36232e-01],
        [2.84144e-03, 5.87304e-01, 4.09855e-01]])}

In [48]:
m[:,1]=0
a = ds.m_fit_and_compute_consensus(m, t, k, 1)
a


(array([[0.47629916, 0.07265595, 0.45104489],
        [0.49253281, 0.30742989, 0.2000373 ],
        [0.18053211, 0.01019056, 0.80927733],
        [0.30813743, 0.01148725, 0.68037531],
        [0.47343758, 0.01165636, 0.51490606],
        [0.47629916, 0.07265595, 0.45104489],
        [0.77143334, 0.05132708, 0.17723958],
        [0.47629916, 0.07265595, 0.45104489],
        [0.28233425, 0.00169881, 0.71596694],
        [0.47629916, 0.07265595, 0.45104489],
        [0.63808478, 0.0642833 , 0.29763192],
        [0.64434392, 0.01047723, 0.34517886],
        [0.77143334, 0.05132708, 0.17723958],
        [0.36105569, 0.34123747, 0.29770684],
        [0.47629916, 0.07265595, 0.45104489],
        [0.62024004, 0.25568139, 0.12407857],
        [0.1915386 , 0.74072739, 0.06773401],
        [0.1915386 , 0.74072739, 0.06773401],
        [0.36105569, 0.34123747, 0.29770684],
        [0.47343758, 0.01165636, 0.51490606],
        [0.30813743, 0.01148725, 0.68037531],
        [0.64434392, 0.01047723, 0

In [49]:
d = smoc.m_fit(m, t, k, w, t_C)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

{'tau': array([0.00140936, 0.104596  , 0.893995  ]),
 'pi': array([[9.99992e-01, 3.75113e-06, 3.75113e-06],
        [1.14160e-02, 9.36988e-01, 5.15958e-02],
        [2.40188e-01, 3.19697e-01, 4.40114e-01]])}

In [50]:
d2 = smoc.m_compute_consensus(m, t, k, w, data=d)
d2

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'w': 20, 't': 100, 'a': 500, 'k': 3, 't_A': array([  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   3,
         3,   3,   4,   4,   4,   4,   4,   5,   5,   5,   5,   5,   6,
         6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,   8,   8,
         8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,  11,
        11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
        14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,
        16,  16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,
        19,  19,  19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,
        21,  22,  22,  22,  22,  22,  23,  23,  23,  23,  23,  24,  24,
        24,  24,  24,  25,  25,  25,  25,  25,  26,  26,  26,  26,  26,
        27,  27,  27,  27,  27,  28,  28,  28,  28,  28,  29,  29,  29,
        29,  29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,
        34,  35,  3

(array([[6.55976e-23, 6.56065e-04, 9.99344e-01],
        [8.89057e-23, 1.61476e-02, 9.83852e-01],
        [9.76045e-29, 6.47655e-05, 9.99935e-01],
        [4.76799e-23, 2.62588e-05, 9.99974e-01],
        [2.32911e-17, 1.06462e-05, 9.99989e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [1.56611e-11, 1.07898e-04, 9.99892e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [1.69187e-17, 4.25846e-07, 1.00000e+00],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [3.20557e-17, 2.66091e-04, 9.99734e-01],
        [1.13774e-11, 4.31629e-06, 9.99996e-01],
        [1.56611e-11, 1.07898e-04, 9.99892e-01],
        [1.77794e-28, 3.89072e-02, 9.61093e-01],
        [6.55976e-23, 6.56065e-04, 9.99344e-01],
        [4.38498e-17, 6.61015e-03, 9.93390e-01],
        [1.26571e-28, 5.03000e-01, 4.97000e-01],
        [1.26571e-28, 5.03000e-01, 4.97000e-01],
        [1.77794e-28, 3.89072e-02, 9.61093e-01],
        [2.32911e-17, 1.06462e-05, 9.99989e-01],
        [4.76799e-23